# Functions & Variables

## CNN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os


class_to_name = {}
dataset_path = r"..\..\dataset\mainDataset"

epoch_amount = 250
batch_size = 32
rotation = 5
terminate_epoch = 125  
dataset = "mainDataset/"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device) 
print(torch.cuda.get_device_name())

charDict = {}
for i, letter in enumerate( sorted( os.listdir( dataset_path + "/train" ) ) ):
    charDict[i] = letter


num_classes = len(charDict)

print(f"{charDict}\n{num_classes}")


class iztechCNN(nn.Module):
    def __init__(self, num_classes = num_classes, image_size = 32 , kernel_sizes = (3, 3, 3) ):
        super(iztechCNN,self).__init__()
        
        final_kernel = 128
        
        self.FC_input = image_size * image_size * final_kernel // (2**3)**2 

        size_1 = kernel_sizes[0]
        size_2 = kernel_sizes[1]
        size_3 = kernel_sizes[2]

        self.conv1 = nn.Conv2d(1, final_kernel//4,               kernel_size=size_1,    padding = (size_1 -1 ) // 2)
        self.conv2 = nn.Conv2d(final_kernel//4, final_kernel//2, kernel_size=size_2,    padding = (size_2 -1 ) // 2)
        self.conv3 = nn.Conv2d(final_kernel//2, final_kernel,    kernel_size=size_3,    padding = (size_3 -1 ) // 2)

        self.pool = nn.MaxPool2d(2,2)

        self.dropout = nn.Dropout(0.3)          

        self.fc1 = nn.Linear(self.FC_input, final_kernel)
        self.fc2 = nn.Linear(final_kernel, num_classes)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.view(-1,self.FC_input)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
       
        return self.fc2(x)

cuda
NVIDIA GeForce RTX 3050 6GB Laptop GPU
{0: '(', 1: ')', 2: '+', 3: '0o', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5s', 9: '6', 10: '7', 11: '8', 12: '9g', 13: '[', 14: ']', 15: 'a', 16: 'c', 17: 'dot', 18: 'e', 19: 'horizontal_line', 20: 'n', 21: 'p', 22: 'r', 23: 'sqrt', 24: 't', 25: 'vertical_line', 26: 'x'}
27


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

val_transform = transforms.Compose([
    transforms.Resize((32,32),interpolation=transforms.InterpolationMode.NEAREST),
    transforms.Grayscale(num_output_channels=1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


## Process Functions

In [114]:
from typing import List, Dict
import numpy as np
import cv2
from PIL import Image

chosen_model = 17

with open(f"./model_classes/charcnn_{chosen_model}_classes.txt","r+") as f:
    data = f.read()

charDict = {}

for line in data.split("\n")[:-1]:
    line = line.split(" ")
    charDict[int(line[0])] = line[1]
num_classes = len(charDict)

class Symbol:
    def __init__(self,char,confidence=None,center=None,size=None):
        self.char = char
        self.confidence = confidence
        self.x, self.y = center
        self.w, self.h = size
        self.used = False

    def isLine(self):
        return "line" in self.char
    
    def __repr__(self): 
        return f"Symbol('{self.char}', ({self.x}, {self.y}), {self.w}x{self.h})"
    
    def __eq__(self, other):
        if not isinstance(other, Symbol):
            return False
        return (self.char, self.x, self.y, self.w, self.h) == (other.char, other.x, other.y, other.w, other.h)

    def __hash__(self):
        return hash((self.char, self.x, self.y, self.w, self.h))

    def __repr__(self):
        return f"{self.char} ({self.x}, {self.y})"
        return f" Symbol Object (char: {self.char}  x: {self.x}  y: {self.y}  w: {self.w}  h: {self.h}) "                                 #f"Symbol('{self.char}', ({self.x}, {self.y}), {self.w}x{self.h})"

symbol_dict = {}
symbols = []

##               PROCESS FUNCTIONS
def SquareRoot(symbols: List[Symbol]):

    used = {}
    

    sqrt_symbols = sorted([s for s in symbols if s.char == "sqrt"], key = lambda s: s.w* s.h,reverse=True)
    if not sqrt_symbols:
        print("No square root symbols found.\n")
        return
    
    print(f"SQRT Symbols: {sqrt_symbols}\n")
    
    i = 0

    while i< len(sqrt_symbols):
        symbol_1 = sqrt_symbols[i]
        symbols_inside = []
        symbol_1.used = True
        print(f"Processing square root symbol: {symbol_1}\n")

        x,y,w,h = symbol_1.x, symbol_1.y, symbol_1.w, symbol_1.h 
        x1_1, x1_2 = x - w//2, x + w//2
        y1_1, y1_2 = y - h//2, y + h//2
        
        j = 0
        while j < len(symbols):
            symbol_2 = symbols[j]

            if symbol_2 is symbol_1:
                symbols.pop(j)
                j += 1
                continue
            if symbol_2.used:
                j += 1
                continue

            x2, w2 = symbol_2.x, symbol_2.w

            if (x1_1 <= x2 <= x1_2 and w2 < w) and (y1_1 <= symbol_2.y <= y1_2 + h//10 and symbol_2.h < h):
                if symbol_2.char == "sqrt":
                    sqrt_symbols.remove(symbol_2)
                    
                print(f"Symbol {symbol_2} is inside the square root")
                # if the symbol is inside the square root
                if symbol_2 not in used:
                    symbols.pop(j)
                    symbols_inside.append(symbol_2)
                    #symbol_2.used = True
                    used[symbol_2] = True
                else:   # Actually this should not happen since we are checking if the symbol is used in the "if" statement above but just in case
                    j += 1
                    continue
            else:
                j += 1
        print()

        print(f"Inside Symbols: {symbols_inside}")
        new_char = f"√( {Process(symbols_inside)} )"

        symbols.append(Symbol(new_char, confidence=100.0, center=(symbol_1.x, symbol_1.y + h//20 ), size=(symbol_1.w, symbol_1.h +  h//10)))
        i += 1
        

def Exponential(symbols: List[Symbol]):
    return


def LetterI(symbols):
    return symbols
def EqualSign(symbols):
    return symbols
def DivisionSign(symbols):
    return symbols


def SortByX(symbols: List[Symbol]):

    return sorted(symbols, key=lambda s: s.x)

def SortDivisionLinesBySize(symbols: List[Symbol]):
    return sorted(
        [s for s in symbols if "horizontal_line" in s.char],
        key=lambda s: s.h * s.w,
        reverse=True
    )

def ProcessLine(symbols: List[Symbol]):
    symbols = SortByX(symbols)

    result = ""

    for s in symbols:
        result += f"{s.char}"

    return result

def ProcessDivision(div_symbol, above, below):
    
    print(f"Above: {above}")
    print(f"Below: {below}")
    
    above_line = Process(above)
    below_line = Process(below)
    

    result = f"(({above_line}) / ({below_line}))"

    return Symbol(result, confidence=100.0,center=(div_symbol.x, div_symbol.y), size=(div_symbol.w, div_symbol.h))

def Process(symbols):

    new_symbols = []
    used = set()

    
    SquareRoot(symbols)
    division_lines = SortDivisionLinesBySize(symbols)
    
    for div_symbol in division_lines:
        if div_symbol in used:
            continue
        
        tolerance = div_symbol.w // 8
        div_left = div_symbol.x - div_symbol.w // 2 - tolerance
        div_right = div_symbol.x + div_symbol.w // 2 + tolerance

        above = []
        below = []

        for s2 in symbols:
            if s2 is div_symbol or s2 in used:
                continue
            if div_left <= s2.x <= div_right:
                if s2.y < div_symbol.y:
                    above.append(s2)
                elif s2.y > div_symbol.y:
                    below.append(s2)

        if not above or not below:
            minus = Symbol("-", 0.0, (div_symbol.x, div_symbol.y), (div_symbol.w, div_symbol.h))
            new_symbols.append(minus)
            used.add(div_symbol)
            continue

        # This is a recursive function
        new_symbol = ProcessDivision(div_symbol, above, below)

        new_symbols.append(new_symbol)
        used.update(above + below + [div_symbol])

    # sorting of remainings
    remaining_symbols = [s for s in symbols if s not in used]
    #print(f"Process output {ProcessLine(SortByX(new_symbols + remaining_symbols))}")
    return ProcessLine(SortByX(new_symbols + remaining_symbols))



DL_model = iztechCNN(num_classes=num_classes)
DL_model.load_state_dict(torch.load(f"./models/charcnn_{chosen_model}.pth",map_location="cuda",weights_only=True))
DL_model.eval()




iztechCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=27, bias=True)
)

## Test Codes

In [118]:
img_paths = [ f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_0.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_1.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_2.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_3.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_4.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_5.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_6.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_7.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_8.png"
             ,f"E:\Python_Projeler\ComputerVisionProjects\FinalProject\codes\ModelCodes\TestImages\sqrt_9.png"
             ]           #f"./TestImages/ToStringTests_1.png", f"./TestImages/ToStringTests_2.png", f"./TestImages/deneme_4.png", f"./TestImages/exponential_test.png" ] # 


Processes = []
for img_path in img_paths:
    symbols = []

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    blur = cv2.GaussianBlur(img,(3,3),1)

    _, thresh = cv2.threshold(blur,0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        max_edge = max(w,h)

        drawn_contour = np.zeros_like(thresh)

        cv2.drawContours(drawn_contour, [contour],-1,255, thickness=cv2.FILLED)

        drawn_contour = cv2.bitwise_and(thresh, thresh, mask = drawn_contour)


        blank = np.zeros((max_edge,max_edge))

        x1,x2 = int( (max_edge - h) / 2 ), int( (max_edge + h) / 2 )
        y1,y2 = int( (max_edge - w) / 2 ), int( (max_edge + w) / 2 )

        blank[x1:x2,y1:y2] = drawn_contour[y:y+h, x:x+w]
        blank = cv2.copyMakeBorder(blank,2,2,2,2,borderType=cv2.BORDER_CONSTANT,value=0)
        
        letter = Image.fromarray(blank.astype(np.uint8))

        letter = val_transform(letter)
        letter = letter.float()
        letter = letter.unsqueeze(0)
        

        with torch.no_grad():
            output = DL_model(letter)
            _, predicted = torch.max(output,1)

        charInt = predicted.tolist()[0]
        char = charDict[charInt]
        new_symbol = Symbol(char,center = (x + w//2 , y + h//2 ), size= (w , h))
        print(new_symbol)
        symbols.append(new_symbol)
    
    Processes.append(Process(symbols))
print(*Processes,sep="\n")
print("\n")



6 (183, 273)
horizontal_line (191, 178)
5s (208, 124)
sqrt (159, 190)
SQRT Symbols: [sqrt (159, 190)]

Processing square root symbol: sqrt (159, 190)

Symbol 6 (183, 273) is inside the square root
Symbol horizontal_line (191, 178) is inside the square root
Symbol 5s (208, 124) is inside the square root

Inside Symbols: [6 (183, 273), horizontal_line (191, 178), 5s (208, 124)]
No square root symbols found.

Above: [5s (208, 124)]
Below: [6 (183, 273)]
No square root symbols found.

No square root symbols found.

6 (183, 273)
horizontal_line (191, 178)
e (703, 195)
t (434, 196)
sqrt (677, 193)
5s (208, 124)
sqrt (159, 190)
SQRT Symbols: [sqrt (159, 190), sqrt (677, 193)]

Processing square root symbol: sqrt (159, 190)

Symbol 6 (183, 273) is inside the square root
Symbol horizontal_line (191, 178) is inside the square root
Symbol 5s (208, 124) is inside the square root

Inside Symbols: [6 (183, 273), horizontal_line (191, 178), 5s (208, 124)]
No square root symbols found.

Above: [5s (20